In [20]:
import json
import os
from datetime import datetime, timezone

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

DATA_PATH = "../claude/source_data/erics-clothes/wardrobe_data.json"
SHEET_TITLE = "Wardrobe Inventory"
SHARE_WITH_EMAIL = "eric@emelz.com"
CREDS_PATH = "../credentials.json"  # Downloaded from Google Cloud Console
TOKEN_PATH = "../token.json"        # Cached user token after first auth

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"    
]

In [13]:
with open(DATA_PATH, "r") as f:
    data = json.load(f)

In [16]:
flow = InstalledAppFlow.from_client_secrets_file(CREDS_PATH, SCOPES)

In [17]:
creds = flow.run_local_server(port=0)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=583771278292-btvhp3f3m31e6bkctn8cprgi6ruddps7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60306%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=5jjARW87IitQQBquNNnaiOTGYqb3e5&access_type=offline


In [21]:
with open(TOKEN_PATH, "w") as f:
    f.write(creds.to_json())

In [22]:
sheets = build("sheets", "v4", credentials=creds)

In [23]:
drive = build("drive", "v3", credentials=creds)

In [24]:
def load_items(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)

    items = data.get("items", [])
    rows = []

    # Header:
    header = [
        "ID",
        "Title",
        "Category",
        "Filename",
        "Slug",
        "Thumbnail",
        "Image",
        "Tags",
        "Notes",
        "Created (UTC ISO8601)"
    ]

    for it in items:
        created_ts = it.get("created_date")
        if isinstance(created_ts, (int, float)):
            # Some values may include fractional seconds
            created_dt = datetime.fromtimestamp(created_ts, tz=timezone.utc)
            created_iso = created_dt.isoformat().replace("+00:00", "Z")
        else:
            created_iso = ""

        tags = it.get("tags", [])
        if isinstance(tags, list):
            tags_text = ", ".join(tags)
        else:
            tags_text = str(tags) if tags is not None else ""

        rows.append([
            it.get("id", ""),
            it.get("title", ""),
            it.get("category", ""),
            it.get("filename", ""),
            it.get("slug", ""),
            it.get("thumbnail", ""),
            it.get("image", ""),
            tags_text,
            it.get("notes", ""),
            created_iso
        ])
    return header, rows

In [26]:
header, rows = load_items(DATA_PATH)

In [29]:
spreadsheet_body = {"properties": {"title": SHEET_TITLE}}

In [30]:
created = sheets.spreadsheets().create(body=spreadsheet_body, fields="spreadsheetId").execute()

In [31]:
spreadsheet_id = created['spreadsheetId']
spreadsheet_id

'1AFA5O-9eIebN4a12zKZWXInqsas49FxKiBTjAigRy4Q'

In [32]:
values = [header] + rows

In [33]:
sheets.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id,
    range="Sheet1!A1",
    valueInputOption="RAW",
    body={"values": values}
).execute()

{'spreadsheetId': '1AFA5O-9eIebN4a12zKZWXInqsas49FxKiBTjAigRy4Q',
 'updatedRange': 'Sheet1!A1:J80',
 'updatedRows': 80,
 'updatedColumns': 10,
 'updatedCells': 800}

In [34]:
requests = [
    {
        "updateSheetProperties": {
            "properties": {"sheetId": 0, "gridProperties": {"frozenRowCount": 1}},
            "fields": "gridProperties.frozenRowCount"
        }
    },
    {
        "autoResizeDimensions": {
            "dimensions": {"sheetId": 0, "dimension": "COLUMNS", "startIndex": 0, "endIndex": len(header)}
        }
    }
]

In [35]:
sheets.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={"requests": requests}).execute()

{'spreadsheetId': '1AFA5O-9eIebN4a12zKZWXInqsas49FxKiBTjAigRy4Q',
 'replies': [{}, {}]}

In [37]:
drive.permissions().create(
    fileId=spreadsheet_id, 
    body={"type": "user", "role": "writer", "emailAddress": SHARE_WITH_EMAIL}, 
    sendNotificationEmail=True
).execute()

{'kind': 'drive#permission',
 'id': '02365986991181812137',
 'type': 'user',
 'role': 'owner'}

In [38]:
len(data['items'])

79

In [39]:
with open("../claude/output/erics-clothes/wardrobe_data.json", "r") as f:
    data1 = json.load(f)

In [40]:
with open("../claude/output/erics-clothes/wardrobe_data2.json", "r") as f:
    data2 = json.load(f)

In [41]:
len(data1['items'])

400

In [42]:
len(data2['items'])

79